In [ ]:
import sys
sys.path.append('../../qick/qick_lib/')
sys.path.append('../soft/')
import Scan
from mkids import *
import matplotlib.pyplot as plt


In [ ]:
board = Scan.getBoard()
if board == 'zcu111':
    bitfile = 'mkids_2x2_kidsim_v1'    
elif board == 'zcu216':
    bitfile = 'mkids_2x2_kidsim_v2'
elif board == 'rfsoc4x2':
    bitfile = 'mkids_v3'
else:
    raise Exception("Do not know what to do with %s"%board)

iKids = 0
iSimu = 0
scan = Scan.Scan(bitfile, iKids=iKids, iSimu=iSimu)

In [ ]:
# Set the mixers so the channel is in the middle of the first nyquist zone.
# Note that the Scan object has the field "fNyquist" which will be set correctly for each board and firmware.
fMixerSimu = 512
fMixerKids = scan.fNyquist/2
print("fNyquist = ",scan.fNyquist)

In [ ]:
# Initialize simulation chain with no resonance
simu = scan.simuChain
simu.analysis.qout(3)
simu.synthesis.qout(3)
simu.alloff()
simu.set_mixer_frequency(fMixerSimu)


In [ ]:
# Initialize the kids chain
kids = scan.kidsChain
# These values for qout work reasonably for loopback.  
kids.analysis.qout(2)
kids.synthesis.qout(2)
kids.set_mixer_frequency(fMixerKids)

In [ ]:
outCh = [30]
fTones = np.array(scan.outCh2Freq(outCh))
fis = np.random.default_rng().uniform(0, 2*np.pi, 1000)
gs = 0.9*np.ones(len(fTones))/len(fTones)
cgs = None
bandwidth = 0.3 #scan.dfChannel
nf = 50
xs = scan.sweep_tones(fTones, fis, gs, cgs, bandwidth, nf, doApplyDelay=True)

In [ ]:
f,x = Scan.sweptTonesToSpectrum(xs, fTones, scan.kidsChain.scanFOffsets)

In [ ]:
plt.plot(f,np.angle(x), '.-')

In [ ]:
plt.plot(f,np.abs(x), '.-')

In [ ]:
def getDelayForOutCh(outCh, df=None, N=50, doProgress=False, plotFit=False, fMixer=700):
    scan.set_mixer(fMixer)
    kids = scan.kidsChain
    qFMixer = scan.get_mixer()
    pfbFs = kids.synthesis.dict['chain']['fs']
    fMinimum = qFMixer-pfbFs/2
    fMaximum = qFMixer+pfbFs/2

    fTone = scan.outCh2Freq(outCh)
    if df is None:
        df = kids.synthesis.fc_ch
    fMin = np.maximum(fMinimum, fTone-df/2)
    fMax = np.minimum(fMaximum, fTone+df/2)
    df = fMax-fMin
    fc = (fMin+fMax)/2
    freqs = np.array([fc])
    fis = np.zeros(1)
    gs = 0.9*np.ones(1)
    kids.set_tones(freqs, fis, gs)
    fOffsets = kids.get_sweep_offsets(df, N)
    xs = kids.sweep_tones(df, N, mean=True, doProgress=doProgress)
    delay,phi0 = measureDelay(fOffsets, xs[:,0], plotFit=plotFit) 
    return delay,xs,fOffsets,fTone,phi0

In [ ]:
delay,xs,fOffsets,fTone,phi0 = getDelayForOutCh(28, df=0.3, N=50, doProgress=True, plotFit=True, fMixer=fMixerKids)
print("fTone =",fTone)
delay28 = delay

In [ ]:
scan.nominalDelay

In [ ]:
scan.get_mixer()

In [ ]:
iCh = 28
outCh = [iCh]
fTones = np.array(scan.outCh2Freq(outCh))
fis = np.random.default_rng().uniform(0, 2*np.pi, 1000)
gs = 0.9*np.ones(len(fTones))/len(fTones)
cgs = None
bandwidth = scan.dfChannel
nf = 50
xs28 = scan.sweep_tones(fTones, fis, gs, cgs, bandwidth, nf, doApplyDelay=False)

In [ ]:
xs28c = applyDelay(fTones, scan.kidsChain.scanFOffsets, xs28, delay28)
f28c,x28c = Scan.sweptTonesToSpectrum(xs28c, fTones, scan.kidsChain.scanFOffsets)
xs28n = applyDelay(fTones, scan.kidsChain.scanFOffsets, xs28, scan.nominalDelay)
f28n,x28n = Scan.sweptTonesToSpectrum(xs28n, fTones, scan.kidsChain.scanFOffsets)
#plt.plot(f28c, np.angle(x28c), '.-', label="best delay")
plt.plot(f28n, np.angle(x28n), '.-', label="nominal delay")
plt.ylabel("Phase [Rad]")
plt.legend()
plt.title("iCh = %d"%iCh)

In [ ]:
iCh = 29
outCh = [iCh]
fTones = np.array(scan.outCh2Freq(outCh))
fis = np.random.default_rng().uniform(0, 2*np.pi, 1000)
gs = 0.9*np.ones(len(fTones))/len(fTones)
cgs = None
bandwidth = scan.dfChannel
nf = 50
xs = scan.sweep_tones(fTones, fis, gs, cgs, bandwidth, nf, doApplyDelay=True)
f,x = Scan.sweptTonesToSpectrum(xs, fTones, scan.kidsChain.scanFOffsets)
plt.plot(f,np.angle(x), '.-')
plt.ylabel("Phase [Rad]")
plt.title("iCh = %d"%iCh)

In [ ]:
iCh = 30
outCh = [iCh]
fTones = np.array(scan.outCh2Freq(outCh))
fis = np.random.default_rng().uniform(0, 2*np.pi, 1000)
gs = 0.9*np.ones(len(fTones))/len(fTones)
cgs = None
bandwidth = scan.dfChannel
nf = 50
xs = scan.sweep_tones(fTones, fis, gs, cgs, bandwidth, nf, doApplyDelay=True)
f,x = Scan.sweptTonesToSpectrum(xs, fTones, scan.kidsChain.scanFOffsets)
plt.plot(f,np.angle(x), '.-')
plt.ylabel("Phase [Rad]")
plt.title("iCh = %d"%iCh)

In [ ]:
outCh = [28, 29, 30]
fTones = np.array(scan.outCh2Freq(outCh))
fis = np.random.default_rng().uniform(0, 2*np.pi, 1000)
gs = 0.9*np.ones(len(fTones))/len(fTones)
cgs = None
bandwidth = scan.dfChannel
nf = 50
xs = scan.sweep_tones(fTones, fis, gs, cgs, bandwidth, nf, doApplyDelay=True)
f,x = Scan.sweptTonesToSpectrum(xs, fTones, scan.kidsChain.scanFOffsets)
plt.plot(f,np.angle(x), '.-')
plt.ylabel("Phase [Rad]")
plt.title("outCh = %s"%str(outCh))

In [ ]:
delay = 0.001
xsc = Scan.applyDelay(fTones, scan.kidsChain.scanFOffsets, xs, delay)
f,x = Scan.sweptTonesToSpectrum(xsc, fTones, scan.kidsChain.scanFOffsets)
plt.plot(f,np.angle(x), '.-')
dtheta = np.angle(x).max()-np.angle(x).min()
plt.ylabel("Phase [Rad]")
plt.title("outCh = %s  delay = %f"%(str(outCh), delay))
print(dtheta)

In [ ]:
outCh = [25, 26, 27, 28, 29, 30, 31]
fTones = np.array(scan.outCh2Freq(outCh))
fis = np.random.default_rng().uniform(0, 2*np.pi, 1000)
gs = 0.9*np.ones(len(fTones))/len(fTones)
cgs = None
bandwidth = scan.dfChannel
nf = 50
xs = scan.sweep_tones(fTones, fis, gs, cgs, bandwidth, nf, doApplyDelay=True)
f,x = Scan.sweptTonesToSpectrum(xs, fTones, scan.kidsChain.scanFOffsets)
plt.plot(f,np.angle(x), '.-')
plt.ylabel("Phase [Rad]")
plt.title("outCh = %s"%str(outCh))

In [ ]:
delay = 0.0005
xsc = Scan.applyDelay(fTones, scan.kidsChain.scanFOffsets, xs, delay)
f,x = Scan.sweptTonesToSpectrum(xsc, fTones, scan.kidsChain.scanFOffsets)
plt.plot(f,np.angle(x), ',-')
dtheta = np.angle(x).max()-np.angle(x).min()
plt.ylabel("Phase [Rad]")
plt.title("outCh = %s  extraDelay = %f"%(str(outCh), delay))
plt.xlabel("frequency [MHz]  fMixer=%f"%scan.get_mixer())
for ch in outCh:
    f = scan.outCh2Freq(ch) + scan.dfChannel/2
    plt.axvline(f, c='r', alpha=0.3)